## Testing Permutation Synchronization using CMU House Sequence dataset

In [3]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linear_sum_assignment
from scipy.io import loadmat
import sys, pathlib

repo_root = pathlib.Path.cwd().parent        
sys.path.append(str(repo_root))

from permsync.perm_sync import perm_sync, error_against_ground_truth

In [4]:
# load data
mat = loadmat("data/house.mat", squeeze_me=True)
data = mat["data"]   # list of m cells, each nx2
scf  = mat["scf"]    # list of m cells, each n×d
N    = len(data)        # number of frames
n    = data[0].shape[0] # landmarks per frame

In [5]:
print(f"Number of frames: {N}, landmarks per image {n}")

Number of frames: 111, landmarks per image 30


In [6]:
rw = [frame[0, :] for frame in data]  # x-coords
cl = [frame[1, :] for frame in data]  # y-coords

# pairewise matching using features
P = [[None for _ in range(N)] for _ in range(N)]

base = np.arange(n)  # [0, 1, ..., 29]

for i in range(N):
    for j in range(i, N):
        m1 = np.vstack((rw[i], cl[i]))  # shape (2, n)
        m2 = np.vstack((rw[j], cl[j]))

        if i == j:
            c = base.copy()
        else:
            # cost matrix (feature distance)
            cormat = np.zeros((n, n))
            for k1 in range(n):
                for k2 in range(n):
                    cormat[k1, k2] = np.linalg.norm(scf[i][k1, :] - scf[j][k2, :])

            # Hungarian algorithm for optimal matching
            row_ind, col_ind = linear_sum_assignment(cormat)
            c = np.zeros(n, dtype=int)
            c[row_ind] = col_ind

            # Handle unmatched nodes by random matching
            confused = np.unique(c)
            cr = np.setdiff1d(base, confused)
            br = np.setdiff1d(base, base[confused])

            kf = np.random.permutation(len(cr))
            c[br] = cr[kf]

        # permutation results
        sorted_idx = np.argsort(c)
        P[i][j] = c
        P[j][i] = base[sorted_idx]

In [7]:
# define method to compute error on P, pairewise matchings
# gt are identity matrix

def compute_relative_error(P, N, n):
    """
    Args:
        P: list of lists; P[i][j] is a permutation array of size n (mapping i → j)
        N: number of frames
        n: number of keypoints per frame

    Returns:
        relative error (float)
    """
    total_errors = 0
    total_entries = 0

    for i in range(N):
        for j in range(i+1, N):
            pred_perm = P[i][j]
            true_perm = np.arange(n)  # ground truth: identity
            error = np.sum(pred_perm != true_perm)
            total_errors += error
            total_entries += n

    return total_errors / total_entries


In [8]:
error_hungarian = compute_relative_error(P, N, n)

In [9]:
print(f"Relative matching error using hungarian: {error_hungarian:.4f}")

Relative matching error using hungarian: 0.2071


### use the pairewise matchings to build T and perform perm_sync

In [10]:
def build_T_tensor(P, N, n):
    T = torch.zeros((N, N, n, n))
    I = torch.eye(n)

    for i in range(N):
        for j in range(N):
            idx = P[i][j]  # This is a permutation of [0,...,n-1]
            T[i, j] = I[:, idx]  # Get the corresponding permutation matrix
    return T

In [11]:
T = build_T_tensor(P, N, n)
tau = perm_sync(T, N, n)

In [12]:
error_sync = compute_relative_error(T, N, n)

In [13]:
print(f"Relative error after synchronizing: {error_sync:.4f}")

Relative error after synchronizing: 0.0333
